In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits as pyfits
import sys, os
import healpy as hp
from calc_BAT_ul import *

from copy import copy
from kapteyn import maputils
import ephem

/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
events = ["GW200112_155838"]

In [8]:
for x in events:
    
    name = x # looping through the event names 
    
    bat_info = "test-GW200112_155838/info.txt"
    lines = open(bat_info).readlines()
    bat_ra = float(lines[25].split()[-1])
    bat_dec = float(lines[26].split()[-1])
    earth_ra = float(lines[23].split()[-1])
    earth_dec = float(lines[24].split()[-1])
    earth_rad = 69. # Earth radius for Swift BAT
    t0 = float(lines[8].split()[-1]) # LIGO trigger time 
    
    # loading the 1-s BAT ligtcurve
    lc_path = "test-GW200112_155838/1s_rate_sort.lc"
    lc_1s = pyfits.open(lc_path)
    counts_1s = lc_1s[1].data['COUNTS']
    time_1s = lc_1s[1].data['TIME'] - t0
    rate_1s = counts_1s 

    # starting the light curve from -1 to 30 to match GBM search window
    index = [i for i,x in enumerate(time_1s) if x>=-1 and x<=30]
    time_new = time_1s[index]
    counts_new = counts_1s[index]
    rate_new = rate_1s[index]
    std = np.std(rate_new) # std from the 0.5 second lightcurve
    
    Ndets_tot = 32768.0 # total dets
    Ndets_active = float(lines[36].split()[-1]) # current number of active dets
    Ndet_ratio = Ndets_active / Ndets_tot

    IDs = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33]
    rate_std = std # the std of rates from the LC
    rate_upper_limit = 5*rate_std
    # print("5sigma rate upper limit: ", rate_upper_limit)
    
    ul_5sigma = [] 
    
    for i in IDs:
        grid_id = i 
        # print("grid_id: ", grid_id)

        # using energy bin 15-350 and ignoring 350-500
        chan_low = 0 
        chan_hi = 3

        # getting the NITRATES DRM table
        drm_tab = get_drm_tab(grid_id)

        # response matrix using selected energy bins and corrected for number of active dets
        drm_matrix = drm_tab['MATRIX'][:,chan_low:(chan_hi+1)] * Ndet_ratio 

        # find the flux that gives an expected rate equal to the rate upper limit
        flux_upper_limit = rate2band_eflux(rate_upper_limit, drm_matrix,\
                                       drm_tab['ENERG_LO'], drm_tab['ENERG_HI'],\
                                       alpha, beta, Epeak, flux_elo, flux_ehi)
        #print("5-sigma flux upper limit [erg/cm2/s]: ", flux_upper_limit)
        ul_5sigma.append(flux_upper_limit)
        np.save(os.path.join('test-GW200112_155838', name + "_BAT_gridID_ul.npy"), ul_5sigma)
        

/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:543: UserWarning: 0.0 180.0 -180.0 180.0
  pmin / dtor, pmax / dtor, mmin / dtor, mmax / dtor
/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:658: UserWarning: The interval between parallels is 30 deg -0.00'.
  vdeg, varcmin
/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:666: UserWarning: The interval between meridians is 30 deg -0.00'.
  vdeg, varcmin


<Figure size 432x288 with 0 Axes>

<Figure size 612x388.8 with 0 Axes>